## Compare district aggregations: using rio.clip and using zonal_stats

In [1]:
%cd ../..

/home/u/amine.barkaoui/anticipatory-action


In [2]:
import xarray as xr

from config.params import Params

from AA.analytical import calculate_forecast_probabilities
from AA.helper_fns import (
    get_coverage,
    read_observations,
    read_forecasts,
    aggregate_by_district,
)

from hip.analysis.analyses.drought import get_accumulation_periods
from hip.analysis.aoi.analysis_area import AnalysisArea
from hip.analysis.compute.utils import start_dask, persist_with_progress_bar

In [3]:
client = start_dask(n_workers=1)
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:42975
INFO:distributed.scheduler:  dashboard at:  /user-redirect/proxy/8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37013'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33277', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33277
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:49416
INFO:distributed.scheduler:Receive client connection: Client-d576e16c-ef5f-11ef-81d6-0a0abee40dbb
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:49418


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user-redirect/proxy/8787/status,
Dashboard: /user-redirect/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 15.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42975,Workers: 1
Dashboard: /user-redirect/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 15.20 GiB
Comm: tcp://127.0.0.1:33277,Total threads: 4
Dashboard: /user-redirect/proxy/40793/status,Memory: 15.20 GiB
Nanny: tcp://127.0.0.1:37013,


INFO:distributed.core:Event loop was unresponsive in Nanny for 4.39s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 4.38s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 4.62s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 4.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 4.43s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause tim

#### Get probabilities at pixel level for Mozambique - October

In [4]:
country = "MOZ"
index = "SPI"
issue = "10"

In [5]:
params = Params(iso=country, index=index)

In [6]:
area = AnalysisArea.from_admin_boundaries(
    iso3=params.iso.upper(),
    admin_level=2,
    resolution=0.25,
    datetime_range=f"1981-01-01/{params.calibration_year}-06-30",
)

gdf = area.get_dataset([area.BASE_AREA_DATASET])

INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/CountriesAndSubdivisions?iso3=MOZ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=170 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900948 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900949 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900950 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900951 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900952 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0=170&admCode=900953 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.vam.wfp.org/geodata/GetGeoAdmins?adm0

In [7]:
# Observations data reading
observations = read_observations(
    area,
    f"{params.data_path}/data/{params.iso}/zarr/{params.calibration_year}/obs/observations.zarr",
)
observations

INFO:root:Reading of observations from precomputed zarr...


<xarray.DataArray 'band' (time: 15156, latitude: 67, longitude: 44)> Size: 357MB
dask.array<open_dataset-band, shape=(15156, 67, 44), dtype=float64, chunksize=(1, 67, 44), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 536B -10.38 -10.62 -10.88 ... -26.62 -26.88
  * longitude    (longitude) float64 352B 30.12 30.38 30.62 ... 40.62 40.88
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 121kB 1981-01-01 ... 2022-06-30
Attributes:
    nodata:   nan

In [8]:
# Forecasts data reading
forecasts = read_forecasts(
    area,
    issue,
    f"{params.data_path}/data/{params.iso}/zarr/{params.calibration_year}/{issue}/forecasts.zarr",
    update=False,
)
forecasts

INFO:root:Reading of forecasts from precomputed zarr...


<xarray.DataArray 'tp' (time: 8702, ensemble: 51, latitude: 67, longitude: 44)> Size: 5GB
dask.array<getitem, shape=(8702, 51, 67, 44), dtype=float32, chunksize=(8702, 1, 67, 44), chunktype=numpy.ndarray>
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    issue        (time) <U7 244kB dask.array<chunksize=(8702,), meta=np.ndarray>
  * latitude     (latitude) float64 536B -10.38 -10.62 -10.88 ... -26.62 -26.88
  * longitude    (longitude) float64 352B 30.12 30.38 30.62 ... 40.62 40.88
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 70kB 1981-10-01 1981-10-02 ... 2022-04-30
Attributes:
    nodata:   nan
    units:    mm

In [9]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts,
    params.start_season,
    params.end_season,
    params.min_index_period,
    params.max_index_period,
)

In [10]:
# Run workflow for one issue month
probabilities, probabilities_bc, anomaly_obs = {}, {}, {}

for period_name, period_months in accumulation_periods.items():

    probs, probs_bc, obs_values, _ = calculate_forecast_probabilities(
        forecasts,
        observations,
        params,
        period_months,
        issue,
    )

    probabilities[period_name] = probs
    probabilities_bc[period_name] = probs_bc
    anomaly_obs[period_name] = obs_values

INFO:root:Completed accumulation
/envs/shared/hdc/lib/python3.10/site-packages/hdc/algo/accessors.py:536: RuntimeWarning: invalid value encountered in cast
  dask_gufunc_kwargs={"meta": self._obj.data.astype(dtype)},
INFO:root:Completed anomaly
INFO:root:Completed bias correction
INFO:root:Completed probabilities
INFO:root:Completed categorical observations
INFO:root:Completed probabilities with bias correction
INFO:distributed.utils_perf:full garbage collection released 96.00 MiB from 268904 reference cycles (threshold: 9.54 MiB)
INFO:root:Completed accumulation
/envs/shared/hdc/lib/python3.10/site-packages/hdc/algo/accessors.py:536: RuntimeWarning: invalid value encountered in cast
  dask_gufunc_kwargs={"meta": self._obj.data.astype(dtype)},
INFO:root:Completed anomaly
INFO:root:Completed bias correction
INFO:root:Completed probabilities
INFO:root:Completed categorical observations
INFO:root:Completed probabilities with bias correction
INFO:root:Completed accumulation
/envs/shared/hd

In [11]:
def concat_along_index(da_dict):
    da = xr.concat(da_dict.values(), dim="index")
    return da.assign_coords({"index": ("index", list(da_dict.keys()))})


probabilities = concat_along_index(probabilities)
probabilities_bc = concat_along_index(probabilities_bc)
anomaly_obs = concat_along_index(anomaly_obs)

####  Aggregate using **rio.clip**

Running time estimate: 1min

In [61]:
%%time
probs_district_rio = aggregate_by_district(probabilities, gdf, params)

CPU times: user 59.8 s, sys: 1.05 s, total: 1min
Wall time: 1min 1s


In [62]:
%%time
probs_bc_district_rio = aggregate_by_district(probabilities_bc, gdf, params)

CPU times: user 59.4 s, sys: 953 ms, total: 1min
Wall time: 59.4 s


In [63]:
%%time
obs_district_rio = aggregate_by_district(anomaly_obs, gdf, params)

CPU times: user 55.2 s, sys: 752 ms, total: 55.9 s
Wall time: 55.2 s


#### Aggregate using **zonal_stats**

Running time estimate: 12s (5x faster)

In [12]:
districts_to_exclude = [
    "Cidade_De_Pemba",
    "Ibo",
    "Pemba",
    "Cidade_De_Xai-Xai",
    "Cidade_De_Inhambane",
    "Maxixe",
    "Cidade_De_Chimoio",
    "Cidade_Da_Matola",
    "Cidade_De_Nampula",
    "Ilha_De_Moçambique",
    "Cidade_De_Lichinga",
    "Maquival",
]

In [39]:
zone_ids = gdf.loc[~gdf.Name.isin(districts_to_exclude)].Name.values

In [40]:
zone_ids, zones = area._resolve_zones(
    probabilities.isel(category=0, index=0), zone_ids=zone_ids
)

In [53]:
%%time
stacked = probabilities.rename({"year": "time"}).stack(
    multi_index=["category", "index"]
)
grouped = stacked.groupby("multi_index")
probs_district_zonal = grouped.map(
    lambda da: area.zonal_stats(
        da.squeeze(["multi_index"]), stats=["mean"], zone_ids=zone_ids, zones=None
    ).to_xarray()["mean"]
).rename({"zone": "district"})

INFO:distributed.utils_perf:full garbage collection released 30.42 MiB from 13535 reference cycles (threshold: 9.54 MiB)


CPU times: user 13.1 s, sys: 464 ms, total: 13.6 s
Wall time: 12.8 s


In [56]:
%%time
stacked = probabilities_bc.rename({"year": "time"}).stack(
    multi_index=["category", "index"]
)
grouped = stacked.groupby("multi_index")
probs_bc_district_zonal = grouped.map(
    lambda da: area.zonal_stats(
        da.squeeze(["multi_index"]), stats=["mean"], zone_ids=zone_ids, zones=None
    ).to_xarray()["mean"]
).rename({"zone": "district"})

CPU times: user 12 s, sys: 387 ms, total: 12.4 s
Wall time: 11.7 s


In [59]:
%%time
obs_district_zonal = (
    anomaly_obs.groupby("index")
    .map(
        lambda da: area.zonal_stats(
            da.squeeze("index"), stats=["mean"], zone_ids=zone_ids, zones=None
        ).to_xarray()["mean"]
    )
    .rename({"zone": "district"})
)

CPU times: user 4.23 s, sys: 170 ms, total: 4.4 s
Wall time: 4.14 s


In [68]:
%%time
# Unstack dimensions
probs_district_zonal = probs_district_zonal.unstack("multi_index")
probs_bc_district_zonal = probs_bc_district_zonal.unstack("multi_index")
obs_district_zonal = obs_district_zonal.unstack("multi_index")

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


#### Compare both methods

Identical results (relative tol: 1e-7, absolute tol: 0)

In [73]:
# Harmonize dimensions / coordinates sorting
probs_district_zonal = (
    probs_district_zonal.sel(
        district=probs_district_rio.district,
        index=probs_district_rio.index,
        category=probs_district_rio.category,
    )
    .rename({"time": "year"})
    .transpose(*probs_district_rio.dims)
)
probs_bc_district_zonal = probs_bc_district_rio.sel(
    district=probs_bc_district_rio.district,
    index=probs_bc_district_rio.index,
    category=probs_bc_district_rio.category,
).rename({"time": "year"})
obs_district_zonal = obs_district_rio.sel(
    district=obs_district_rio.district,
    index=obs_district_rio.index,
)

In [83]:
xr.testing.assert_allclose(probs_district_rio, probs_district_zonal)

In [76]:
xr.testing.assert_allclose(probs_bc_district_rio, probs_bc_district_zonal)

In [75]:
xr.testing.assert_allclose(obs_district_rio, obs_district_zonal)

#### Check datasets to visualize the indices / categories / districts that we compare

In [84]:
probs_district_rio

<xarray.DataArray 'tp' (district: 146, index: 11, year: 41, category: 3)> Size: 2MB
array([[[[0.14857143, 0.2       , 0.24      ],
         [0.33714286, 0.37714286, 0.45714286],
         [0.07428571, 0.08571429, 0.14857143],
         ...,
         [0.06857143, 0.08857143, 0.11142857],
         [0.05142857, 0.07714286, 0.12571429],
         [0.1       , 0.13428571, 0.18857143]],

        [[0.11428571, 0.21714286, 0.31428571],
         [0.18857143, 0.25142857, 0.32571429],
         [0.18285714, 0.20571429, 0.24571429],
         ...,
         [0.25      , 0.27571429, 0.31285714],
         [0.1       , 0.16      , 0.21571429],
         [0.12285714, 0.19142857, 0.23142857]],

        [[0.29142857, 0.30857143, 0.34285714],
         [0.22857143, 0.24      , 0.27428571],
         [0.22857143, 0.24571429, 0.26857143],
         ...,
...
         ...,
         [0.05692308, 0.08153846, 0.1       ],
         [0.08615385, 0.12923077, 0.20153846],
         [0.09846154, 0.14923077, 0.18923077]],

        [[0.18153846, 0.2       , 0.22769231],
         [0.24615385, 0.28615385, 0.33538462],
         [0.09230769, 0.15076923, 0.18153846],
         ...,
         [0.13076923, 0.16      , 0.19230769],
         [0.16      , 0.18153846, 0.20307692],
         [0.12923077, 0.17384615, 0.19384615]],

        [[0.16      , 0.19692308, 0.24      ],
         [0.42461538, 0.45538462, 0.52923077],
         [0.12307692, 0.16615385, 0.21538462],
         ...,
         [0.16615385, 0.19692308, 0.22076923],
         [0.17384615, 0.20615385, 0.22692308],
         [0.09230769, 0.11076923, 0.16615385]]]])
Coordinates:
  * category     (category) object 24B 'Severe' 'Moderate' 'Mild'
  * year         (year) int64 328B 1981 1982 1983 1984 ... 2018 2019 2020 2021
  * index        (index) <U3 132B 'JF' 'FM' 'MA' 'ON' ... 'OND' 'NDJ' 'DJF'
    spatial_ref  int64 8B 0
  * district     (district) <U19 11kB 'Ancuabe' 'Balama' ... 'Pebane'

In [85]:
probs_bc_district_rio

<xarray.DataArray 'scen' (district: 146, index: 11, year: 41, category: 3)> Size: 2MB
array([[[[0.        , 0.01142857, 0.06857143],
         [0.10857143, 0.19428571, 0.29142857],
         [0.28571429, 0.29714286, 0.34857143],
         ...,
         [0.02285714, 0.03142857, 0.04285714],
         [0.22857143, 0.25571429, 0.29428571],
         [0.37      , 0.40428571, 0.44714286]],

        [[0.10285714, 0.14285714, 0.24571429],
         [0.16571429, 0.19428571, 0.29714286],
         [0.20571429, 0.23428571, 0.24      ],
         ...,
         [0.17428571, 0.18571429, 0.22      ],
         [0.16571429, 0.19142857, 0.20571429],
         [0.20857143, 0.22      , 0.24571429]],

        [[0.32571429, 0.33142857, 0.34285714],
         [0.24571429, 0.30285714, 0.36      ],
         [0.24571429, 0.24571429, 0.25142857],
         ...,
...
         ...,
         [0.07384615, 0.08153846, 0.1       ],
         [0.17846154, 0.23230769, 0.26384615],
         [0.16769231, 0.23846154, 0.32923077]],

        [[0.21538462, 0.22769231, 0.24307692],
         [0.17230769, 0.18461538, 0.23384615],
         [0.20615385, 0.23692308, 0.29846154],
         ...,
         [0.17076923, 0.2       , 0.23230769],
         [0.17538462, 0.19538462, 0.21461538],
         [0.20769231, 0.22692308, 0.26153846]],

        [[0.04923077, 0.10153846, 0.15076923],
         [0.33230769, 0.41846154, 0.48307692],
         [0.27692308, 0.33846154, 0.41538462],
         ...,
         [0.03538462, 0.06      , 0.12307692],
         [0.23      , 0.25076923, 0.28384615],
         [0.26307692, 0.34230769, 0.43307692]]]])
Coordinates:
  * category     (category) object 24B 'Severe' 'Moderate' 'Mild'
  * year         (year) int64 328B 1981 1982 1983 1984 ... 2018 2019 2020 2021
  * index        (index) <U3 132B 'JF' 'FM' 'MA' 'ON' ... 'OND' 'NDJ' 'DJF'
    spatial_ref  int64 8B 0
  * district     (district) <U19 11kB 'Ancuabe' 'Balama' ... 'Pebane'

In [86]:
obs_district_rio

<xarray.DataArray 'band' (district: 146, index: 11, time: 41)> Size: 527kB
array([[[ -713.71428571,  -478.71428571,   906.71428571, ...,
           -92.14285714, -2485.57142857,  1595.        ],
        [  507.28571429,  -940.57142857,   548.71428571, ...,
         -2406.57142857,  1029.14285714,    69.28571429],
        [  -91.71428571,  -203.14285714,  -100.85714286, ...,
         -2971.71428571,  1409.42857143,   -85.14285714],
        ...,
        [ -305.57142857,  1139.57142857,   -24.85714286, ...,
          1653.14285714, -2287.85714286, -2909.28571429],
        [-1549.14285714,  1034.14285714,   242.        , ...,
          1633.42857143, -2991.85714286,  -790.14285714],
        [ -676.28571429,   144.14285714,   746.42857143, ...,
          1186.57142857, -3259.14285714,   -67.57142857]],

       [[-1095.71428571,  -732.71428571,    87.71428571, ...,
          -409.14285714, -1355.71428571,  1902.71428571],
        [  488.28571429, -1560.14285714,    79.14285714, ...,
         -2883.57142857,  2126.        ,   -73.85714286],
        [  650.14285714,  -614.85714286,    44.        , ...,
         -2702.42857143,  1948.        ,   498.85714286],
...
        [ 2117.33333333,   126.33333333,  -539.33333333, ...,
          -138.66666667,  -391.66666667, -1417.33333333],
        [ 1666.33333333, -1715.66666667,  -310.        , ...,
           505.66666667,  -129.        ,  -737.33333333],
        [ 1659.66666667, -1393.        ,   443.        , ...,
           -52.        ,    45.66666667,  -315.33333333]],

       [[  904.23076923,  -829.        ,   764.15384615, ...,
            16.        , -1027.38461538,   249.30769231],
        [  360.61538462, -1172.69230769,   905.53846154, ...,
         -2141.84615385,  -227.38461538,   190.15384615],
        [  370.15384615, -1581.15384615,   909.53846154, ...,
         -1940.69230769,    24.53846154,   682.        ],
        ...,
        [ 1990.30769231,   195.76923077,  -128.61538462, ...,
           495.        , -1602.84615385, -1591.23076923],
        [ 1951.07692308,  -883.61538462,   487.15384615, ...,
           916.30769231, -1888.15384615,  -540.69230769],
        [ 1794.07692308,  -910.69230769,   751.61538462, ...,
            51.92307692, -1590.76923077,  -369.84615385]]])
Coordinates:
  * time         (time) datetime64[ns] 328B 1981-01-01 1982-01-01 ... 2021-01-01
  * index        (index) <U3 132B 'JF' 'FM' 'MA' 'ON' ... 'OND' 'NDJ' 'DJF'
    spatial_ref  int64 8B 0
  * district     (district) <U19 11kB 'Ancuabe' 'Balama' ... 'Pebane'